In [1]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn import metrics
from tpot import TPOTClassifier

In [2]:
# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
tpot_data = pd.read_csv('simulatedGenex.csv', index_col=False, header=0)
tpot_data =tpot_data.astype(np.float64)
tpot_data = tpot_data.drop(tpot_data.columns[0], axis=1)

features = tpot_data.drop('class', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['class'], random_state=1618)

# Average CV score on the training set was:0.5854060066740823
exported_pipeline = XGBClassifier(learning_rate=1.0, max_depth=5, min_child_weight=7, 
                                  n_estimators=100, nthread=1, subsample=0.4)
tpot_obj= TPOTClassifier()

In [3]:
exported_pipeline.fit(training_features, training_target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=1.0, max_delta_step=0,
       max_depth=5, min_child_weight=7, missing=None, n_estimators=100,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.4)

In [4]:
exported_pipeline.score(testing_features, testing_target)

0.52

In [5]:
# print(exported_pipeline.steps[-1][1].feat_list)
# exported_pipeline.steps[-1][1].feature_importances_
# feat_imp = pd.DataFrame({'feat': exported_pipeline.steps[0][1].feat_list, 
#             'score': exported_pipeline.steps[-1][1].feature_importances_})
# feat_imp.to_csv("featureImpTPOTStand.csv")

In [6]:
results = exported_pipeline.predict_proba(testing_features)

In [7]:
pd.DataFrame({'y':testing_target, 'ypred':results[:,1]}).to_csv("XGBoostPredSim.csv", index=False)

In [8]:
fpr, tpr, thresholds = metrics.roc_curve(testing_target, results[:,1])
metrics.auc(fpr, tpr)

0.5836038961038961

In [9]:
pd.DataFrame({'specificities': fpr, 'sensitivities': tpr, 'auc': metrics.auc(fpr, tpr)}).to_csv("XGBoostROC.csv", index=False)